# **Durée de retour à l'emploi : phase exploratoire**

In [1]:
# nécessité d'importer la librairie xlrd pour permettre à Python de lire Excel (les données INSEE ne sont pas dispobinle en csv)
!pip install xlrd

In [2]:
!pip install openpyxl
# également demandé par Python pour lire Excel

In [3]:
import pandas as pd
import xlrd
import openpyxl
import numpy as np

## 1. Constitution de la BDD OPEN DATA

4 fichiers sont nécessaires à la constitution de cette df :
- 2 indicateurs de dynamisme du marché du travail : taux de chômage et indice de concentration de l'emploi
- 2 codes pour opérer les jointures : code INSEE de la commune (jointure avec les codes postaux) et de la Zone d'emploi (jointure avec les codes communes et le taux de chômage)

**Chargement du fichier de correspondance Code INSEE - Code postal**
- Source : La Poste sur Open Data

> *La base officielle des codes postaux est un jeu de données qui fournit la correspondance entre les codes postaux et les codes INSEE des communes, de France (métropole et DOM), des TOM, ainsi que de MONACO. Le fichier CSV a été mis à jour le 21 mars 2022*

- URL : https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/#resources

In [4]:
code_communes_df = pd.read_excel("laposte_hexasmal_3.xlsx")
code_communes_df.head(10)

,code_commune_insee,nom_de_la_commune,code_postal,ligne_5,libelle_d_acheminement,coordonnees_gps
0,25620,VILLE DU PONT,25650,NaN,VILLE DU PONT,"46.999873398,6.498147193"
1,25624,VILLERS GRELOT,25640,NaN,VILLERS GRELOT,"47.361512085,6.235167025"
2,26001,SOLAURE EN DIOIS,26150,AIX EN DIOIS,SOLAURE EN DIOIS,"44.70587354,5.387770455"
3,26003,ALEYRAC,26770,NaN,ALEYRAC,"44.499150841,4.946083614"
4,26009,ANDANCETTE,26140,NaN,ANDANCETTE,"45.246625023,4.812717032"
5,26011,AOUSTE SUR SYE,26400,NaN,AOUSTE SUR SYE,"44.706856708,5.057320177"
6,26013,ARPAVON,26110,NaN,ARPAVON,"44.371127022,5.273184253"
7,26021,AUTICHAMP,26400,NaN,AUTICHAMP,"44.683240066,4.972677118"
8,26024,BARCELONNE,26120,NaN,BARCELONNE,"44.867074421,5.054652574"
9,26026,BARRET DE LIOURE,26570,NaN,BARRET DE LIOURE,"44.175483103,5.516253783"


In [5]:
code_communes_df.shape

(39193, 6)

In [6]:
# modification et préparation de la table code_communes_df : suppression colonnes inutiles, en-têtes renommées
code_communes_df = code_communes_df.rename(columns = {'code_postal' : 'cp', 'code_commune_insee':'code_commune', 'nom_de_la_commune' : 'commune'})
code_communes_df = code_communes_df[['cp', 'code_commune', 'commune']]
code_communes_df

,cp,code_commune,commune
0,25650,25620,VILLE DU PONT
1,25640,25624,VILLERS GRELOT
2,26150,26001,SOLAURE EN DIOIS
3,26770,26003,ALEYRAC
4,26140,26009,ANDANCETTE
...,...,...,...
39188,98814,98820,OUVEA
39189,98826,98826,POUM
39190,98883,98831,VOH
39191,98818,98833,KOUAOUA


In [7]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
code_communes_df.isna().sum()

cp              0
code_commune    0
commune         0
dtype: int64

In [8]:
code_communes_df.dtypes

cp               int64
code_commune    object
commune         object
dtype: object

**Chargement du fichier base des zones d'emploi Code INSEE - ZE depuis INSEE**
- Source : INSEE - composition communale des zones d'emploi (au format XLSX)
> *Base des zones d'emploi 2020 au 1ᵉʳ janvier 2022. Permet de rattacher chaque commune à sa zone d'emploi, car le taux de chômage actualisé est disponible par zone d'emploi et non par commune*
- URL : https://www.insee.fr/fr/information/4652957

In [9]:
code_ze_df = pd.read_excel("ZE2020_au_01-01-2022.xlsx", skiprows = 4, header = 1, sheet_name = 'Composition_communale')
code_ze_df

,CODGEO,LIBGEO,ZE2020,LIBZE2020,ZE2020_PARTIE_REG,DEP,REG
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse,NaN,01,84
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse,NaN,01,84
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse,NaN,01,84
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône,NaN,01,84
4,01006,Ambléon,8404,Belley,NaN,01,84
...,...,...,...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte,NaN,976,6
34951,97614,Ouangani,601,Mayotte,NaN,976,6
34952,97615,Pamandzi,601,Mayotte,NaN,976,6
34953,97616,Sada,601,Mayotte,NaN,976,6


In [10]:
# modification et préparation de la table code_ze_df : suppression colonnes inutiles, en-têtes renommées
code_ze_df = code_ze_df.rename(columns = {"CODGEO":'code_commune2', 
                                          "LIBGEO" : 'commune2',  
                                          "ZE2020" : 'code_ze',
                                         "LIBZE2020" : 'ze'})
code_ze_df = code_ze_df[["code_commune2", "commune2", "code_ze", "ze"]]
code_ze_df

,code_commune2,commune2,code_ze,ze
0,01001,L'Abergement-Clémenciat,8405,Bourg en Bresse
1,01002,L'Abergement-de-Varey,8405,Bourg en Bresse
2,01004,Ambérieu-en-Bugey,8405,Bourg en Bresse
3,01005,Ambérieux-en-Dombes,8434,Villefranche-sur-Saône
4,01006,Ambléon,8404,Belley
...,...,...,...,...
34950,97613,M'Tsangamouji,601,Mayotte
34951,97614,Ouangani,601,Mayotte
34952,97615,Pamandzi,601,Mayotte
34953,97616,Sada,601,Mayotte


In [11]:
#vérifier le nombre de valeurs manquantes dans chaque colonne
code_ze_df.isna().sum()

code_commune2    0
commune2         0
code_ze          0
ze               0
dtype: int64

In [12]:
code_ze_df.dtypes

code_commune2    object
commune2         object
code_ze           int64
ze               object
dtype: object

**MERGE sur les 2 premiers JDD CP et codes communes et ZE**
> *Constitution d'un seul jeu avec toutes les informations CP, code commune et nom de la commune, code ZE et nom de la ZE. Dans un second temps, on enlevera les colonnes inutiles (gardées pour vérification).*

> *ATTENTION : le 1er JDD comprend 39 193 lignes (codes potaux) alors que le second en a uniquement 34 956 (codes communes et codes ZE). Il faudra donc vérifier quels codes postaux n'ont pas de codes INSEE (par hypothèse les CP d'arrondissements à Paris, Marseille, Lyon*

In [ ]:
#TENTATIVE n°1 : ne marche pas
#ATTENTION : il faut faire la clé à DROITE ou à gauche ?
#code_INSEE_df = pd.merge(code_communes_df, code_ze_df, how = "left", left_on="code_commune", right_on="code_commune2")

In [ ]:
#TENTATIVE n°2 (méthode d'Adrien) : ne marche pas non plus
pd.merge(
    code_communes_df.reset_index(),
    code_ze_df.reset_index(),
    on=['code_commune', 'code_commune2']
)

In [16]:
?pd.merge

Signature:
pd.merge(
    left: 'DataFrame | Series',
    right: 'DataFrame | Series',
    how: 'str' = 'inner',
    on: 'IndexLabel | None' = None,
    left_on: 'IndexLabel | None' = None,
    right_on: 'IndexLabel | None' = None,
    left_index: 'bool' = False,
    right_index: 'bool' = False,
    sort: 'bool' = False,
    suffixes: 'Suffixes' = ('_x', '_y'),
    copy: 'bool' = True,
    indicator: 'bool' = False,
    validate: 'str | None' = None,
) -> 'DataFrame'
Docstring:
Merge DataFrame or named Series objects with a database-style join.

A named Series object is treated as a DataFrame with a single named column.

The join is done on columns or indexes. If joining columns on
columns, the DataFrame indexes *will be ignored*. Otherwise if joining indexes
on indexes or indexes on a column or columns, the index will be passed on.
When performing a cross merge, no column specifications to merge on are
allowed.

Parameters
----------
left : DataFrame
right : DataFrame or named Series
 

In [23]:
#TENTATIVE n°3 : pd.merge(df1, df2, left_on="", right_on="")
code_INSEE_df = pd.merge(code_communes_df, code_ze_df, left_on="code_commune", right_on="code_commune2")

In [24]:
code_INSEE_df.shape

(419, 7)

In [25]:
# Affiche le nombre de valeurs présentes dans chaque colonnes
code_INSEE_df.count()

cp               419
code_commune     419
commune          419
code_commune2    419
commune2         419
code_ze          419
ze               419
dtype: int64

___QUELLE QUE SOIT LA METHODE : impossible de fusionner : opération QUE sur 419 lignes et non a minima les 34 955 lignes du jeu de données Zones d'emploi___

In [26]:
code_INSEE_df

,cp,code_commune,commune,code_commune2,commune2,code_ze,ze
0,20167,2A017,APPIETTO,2A017,Appietto,9401,Ajaccio
1,20110,2A018,ARBELLARA,2A018,Arbellara,9407,Propriano
2,20190,2A026,AZILONE AMPAZA,2A026,Azilone-Ampaza,9401,Ajaccio
3,20119,2A031,BASTELICA,2A031,Bastelica,9401,Ajaccio
4,20133,2A062,CARBUCCIA,2A062,Carbuccia,9401,Ajaccio
...,...,...,...,...,...,...,...
414,20250,2B315,SANTO PIETRO DI VENACO,2B315,Santo-Pietro-di-Venaco,9404,Corte
415,20270,2B328,TOX,2B328,Tox,9405,Ghisonaccia
416,20218,2B332,URTACA,2B332,Urtaca,9403,Calvi
417,20232,2B333,VALLECALLE,2B333,Vallecalle,9402,Bastia
